In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:

import os, sys

def get_dir_n_levels_up(path, n):
    # Go up n levels from the given path
    for _ in range(n):
        path = os.path.dirname(path)
    return path


proj_root = get_dir_n_levels_up(os.path.abspath("__file__"), 2)
sys.path.append(proj_root)

from pathlib import Path
import traceback
from typing import Dict

from liftoff import parse_opts

from opinion_dqn import AgentDQN
from utils import my_logging
from utils.experiment import seed_everything, create_path_to_experiment_folder, build_environment
from utils.generic import convert_namespace_to_dict
import yaml

In [12]:
# softmax in last layer because it scales with N instead of 2^N where N is the nr of agents

# might need to do policy gradient

# next step: Q iteration with action representation

In [13]:
train_env = build_environment()
train_env.action_space.shape[0]

4

In [14]:
experiment_yaml = "2025Jan15-202913_configs"
yaml_path = Path(
    r"D:\Work\repos\RL\phd-rl-algos\dqn\opinion_dynamics\experiments\results"
) / experiment_yaml / "0000_estimator.args_.lin_hidden_out_size_32" / "0" / "cfg.yaml"


with open(yaml_path, 'r') as file:
    config = yaml.safe_load(file)
seed = int(os.path.basename(config["out_dir"]))

seed_everything(seed)

logs_file = os.path.join(config["out_dir"], "experiment_log.log")

logger = my_logging.setup_logger(
    name=config["experiment"],
    # log_file=logs_file,
)

logger.info(f"Starting experiment: {config['full_title']}")

### Setup environments ###
train_env = build_environment()
validation_env = build_environment()

### Setup output and loading paths ###

path_previous_experiments_outputs = None
if "restart_training_timestamp" in config:
    path_previous_experiments_outputs = create_path_to_experiment_folder(
        config,
        config["out_dir"],
        config["restart_training_timestamp"],
    )

experiment_agent = AgentDQN(
    train_env=train_env,
    validation_env=validation_env,
    experiment_output_folder=config["out_dir"],
    experiment_name=config["experiment"],
    resume_training_path=path_previous_experiments_outputs,
    save_checkpoints=True,
    logger=logger,
    config=config
)

logger.info(
    f'Initialized agent with models: {experiment_agent.policy_model}'
)

experiment_agent.train(train_epochs=config["epochs_to_train"])

logger.info(
    f'Finished training experiment: {config["full_title"]}, seed: {config["seed"]}'
)

my_logging.cleanup_file_handlers(experiment_logger=logger)


2025-01-16 11:32:37,219 - opinion_agent_dqn - INFO - Starting experiment: 2025Jan15-202913_configs_estimator.args_.lin_hidden_out_size=32
2025-01-16 11:32:37,219 - opinion_agent_dqn - INFO - Starting experiment: 2025Jan15-202913_configs_estimator.args_.lin_hidden_out_size=32
2025-01-16 11:32:37,220 - opinion_agent_dqn - INFO - Loaded configuration settings.
2025-01-16 11:32:37,220 - opinion_agent_dqn - INFO - Loaded configuration settings.
2025-01-16 11:32:37,222 - opinion_agent_dqn - INFO - Initialized newtworks and optimizer.
2025-01-16 11:32:37,222 - opinion_agent_dqn - INFO - Initialized newtworks and optimizer.
2025-01-16 11:32:37,223 - opinion_agent_dqn - INFO - Initialized agent with models: OpinionNet(
  (fc): Sequential(
    (lin1): Linear(in_features=4, out_features=32, bias=True)
    (relu1): ReLU()
    (lin2): Linear(in_features=32, out_features=32, bias=True)
    (relu2): ReLU()
  )
  (predict_A_b): Sequential(
    (lin3): Linear(in_features=32, out_features=8, bias=True)


d:\Work\repos\RL\phd-rl-algos\dqn\opinion_dynamics\opinion_dqn.py:811: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  actions = torch.FloatTensor(actions)  # Continuous actions


2025-01-16 11:34:28,413 - opinion_agent_dqn - INFO - TRAINING STATS | Frames seen: 200000 | Episode: 400 | Max reward: -65.20954349138614 | Avg reward: -67.70372924618863 | Avg frames (episode): 500.0 | Avg max Q: -5715618.730975764 | Epsilon: 0.2278 | Train epoch time: 0:01:51.154937
2025-01-16 11:34:28,413 - opinion_agent_dqn - INFO - TRAINING STATS | Frames seen: 200000 | Episode: 400 | Max reward: -65.20954349138614 | Avg reward: -67.70372924618863 | Avg frames (episode): 500.0 | Avg max Q: -5715618.730975764 | Epsilon: 0.2278 | Train epoch time: 0:01:51.154937
2025-01-16 11:34:28,414 - opinion_agent_dqn - INFO - Starting validation epoch at t = 200000
2025-01-16 11:34:28,414 - opinion_agent_dqn - INFO - Starting validation epoch at t = 200000
2025-01-16 11:34:54,805 - opinion_agent_dqn - INFO - VALIDATION STATS | Max reward: -67.16648243301412 | Avg reward: -67.82058482099775 | Avg frames (episode): 499.7011952191235 | Avg max Q: -162179605.94268203 | Validation epoch time: 0:00:2

KeyError: 'seed'